In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import uuid
import shortuuid
import scipy.spatial.distance as dist
from enum import Enum
from abc import ABC, abstractmethod


In [2]:
class BuildingType(Enum):
    # define the types of buildings
    # 0: home
    # 1: school
    # 2: groceries
    NONE = '-'
    HOME = 'H'
    SCHOOL = 'S'
    GROCERIES = 'G'
    PAVEMENT = 'P'

In [3]:
BuildingType.HOME.value

'H'

In [4]:
class Node:

    node_list = []

    def __init__(self, subgraph, coordinates: tuple, btype=BuildingType.NONE) -> None:
        self.id = shortuuid.uuid()
        self.coordinates = coordinates
        self.logo = btype.value
        self.element_type = None
        self.node_list.append(self)
        self.subgraph = subgraph
        self.subgraph.add_node(self)

    def __repr__(self) -> str:
        return f"Node({str(self.id)}, {self.coordinates}, {self.logo})"

    def place_on_grid(self, dataframe):
        row,col = self.coordinates
        dataframe[row][col] = self.logo
        return dataframe
    
    def euclidean_distance(self, node):
        return round(dist.euclidean(self.coordinates, node.coordinates), 2)

class Buidling(Node):

    def __init__(self, subgraph, coordinates: tuple, btype: BuildingType) -> None:
        super().__init__(subgraph, coordinates, btype)
        self.element_type = "building"

class Pavement(Node):

    def __init__(self, subgraph, coordinates: tuple, btype=BuildingType.PAVEMENT) -> None:
        super().__init__(subgraph, coordinates, btype)
        self.element_type = "pavement"



In [5]:
# Traffic Graph Class
class TrafficGraph(nx.Graph):

    def __init__(self, element_type: str) -> None:
        super().__init__()
        self.element_type = element_type

    def add_node(self, node: Node) -> None:
        super().add_node(node)
    
    def add_edge(self, source: Node, sinks: list) -> None:
        if not isinstance(sinks, list):
            sinks = [sinks]
        for sink in sinks:
            super().add_edge(source, sink, weight=source.euclidean_distance(sink))


### Create Nodes

In [6]:
pavements = TrafficGraph("pavement")
buildings = TrafficGraph("building")

In [7]:


# create buidlings
groceries = Buidling(buildings, (1,1), BuildingType.GROCERIES)
school = Buidling(buildings, (1,9), BuildingType.SCHOOL)
home = Buidling(buildings, (7,1), BuildingType.HOME)

# create pavements froom groceries to school 
for i in range(2, 9):
    Pavement(pavements, (1,i))

# create pavements from grocieres to home
for i in range(2, 7):
    Pavement(pavements, (i,1))




In [8]:
def initalize_grid(symbol: str='*') -> pd.DataFrame:
    grid = pd.DataFrame(np.chararray((10,10), itemsize=2))
    grid[:] = symbol
    for node in Node.node_list:
        grid = node.place_on_grid(grid)
    return grid

# create a grid
grid = initalize_grid()
grid

,0,1,2,3,4,5,6,7,8,9
0,*,*,*,*,*,*,*,*,*,*
1,*,G,P,P,P,P,P,H,*,*
2,*,P,*,*,*,*,*,*,*,*
3,*,P,*,*,*,*,*,*,*,*
4,*,P,*,*,*,*,*,*,*,*
5,*,P,*,*,*,*,*,*,*,*
6,*,P,*,*,*,*,*,*,*,*
7,*,P,*,*,*,*,*,*,*,*
8,*,P,*,*,*,*,*,*,*,*
9,*,S,*,*,*,*,*,*,*,*


In [9]:
Node.node_list

[Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G),
 Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S),
 Node(GYR9pCcwYY2XY9TxXdDCnM, (7, 1), H),
 Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P),
 Node(PHNeqBRRyKVRqxzooC5AFa, (1, 3), P),
 Node(6yHugKLTsZJ3BKCMmz869Q, (1, 4), P),
 Node(YNWrBX8w6YwPcnuLYwteJA, (1, 5), P),
 Node(ABGXREiYvdtqDDN3HFaWwB, (1, 6), P),
 Node(XQ5uCrbivtNXbyJtwtJirs, (1, 7), P),
 Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P),
 Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P),
 Node(Msmr9nNPXxEGgYFRCwLQnh, (3, 1), P),
 Node(nj4T2NZSGmFnuGWqBgf6kn, (4, 1), P),
 Node(gRrLuGysZCKdBxPLKh9n5M, (5, 1), P),
 Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P)]

In [10]:
# buildings.add_node(school)
# buildings.add_node(home)
# buildings.add_node(groceries)
buildings.nodes
# buildings.node_register

NodeView((Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G), Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S), Node(GYR9pCcwYY2XY9TxXdDCnM, (7, 1), H)))

In [11]:
# find pavements closest to school
min_value = 100
for pavement in pavements.nodes:
    print(pavement.euclidean_distance(groceries), '\t',pavement.id)
    if pavement.euclidean_distance(groceries) < min_value:
        min_value = pavement.euclidean_distance(groceries)
        print(min_value)
        closest = pavement
print(min_value, closest.id)  

1.0 	 dNACFqpPdwNp8qoxYauP82
1.0
2.0 	 PHNeqBRRyKVRqxzooC5AFa
3.0 	 6yHugKLTsZJ3BKCMmz869Q
4.0 	 YNWrBX8w6YwPcnuLYwteJA
5.0 	 ABGXREiYvdtqDDN3HFaWwB
6.0 	 XQ5uCrbivtNXbyJtwtJirs
7.0 	 8rKy9ztEH7z9AMpokagfkJ
1.0 	 23wHU3yz8zvwQ8hRzfpjpH
2.0 	 Msmr9nNPXxEGgYFRCwLQnh
3.0 	 nj4T2NZSGmFnuGWqBgf6kn
4.0 	 gRrLuGysZCKdBxPLKh9n5M
5.0 	 9HqtLUwQb3PHsTgmGHaZA7
1.0 dNACFqpPdwNp8qoxYauP82


In [12]:
def closted_pavements(node: Node, graph: TrafficGraph) -> list:
    distances = [node.euclidean_distance(pavement) for pavement in graph.nodes]
    min_value = min(distances)
    closest = [idx for idx, distance in enumerate(distances) if distance == min_value]
    nodes = [node for node in graph.nodes]
    return [nodes[idx] for idx in closest]

In [13]:
closted_pavements(groceries, pavements)

[Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P),
 Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P)]

In [14]:
closted_pavements(school, pavements)

[Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P)]

In [15]:
buildings.add_edge(school, closted_pavements(school, pavements))
buildings.edges


EdgeView([(Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P))])

In [16]:
buildings.add_edge(groceries, closted_pavements(groceries, pavements))
buildings.add_edge(home, closted_pavements(home, pavements))
buildings.edges

EdgeView([(Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G), Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P)), (Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G), Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P)), (Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P)), (Node(GYR9pCcwYY2XY9TxXdDCnM, (7, 1), H), Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P))])

In [17]:
# connect all pavements with euclidean of 1
for pavement in pavements.nodes:
    for other_pavement in pavements.nodes:
        if pavement.euclidean_distance(other_pavement) == 1:
            pavements.add_edge(pavement, other_pavement)
pavements.edges

EdgeView([(Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P), Node(PHNeqBRRyKVRqxzooC5AFa, (1, 3), P)), (Node(PHNeqBRRyKVRqxzooC5AFa, (1, 3), P), Node(6yHugKLTsZJ3BKCMmz869Q, (1, 4), P)), (Node(6yHugKLTsZJ3BKCMmz869Q, (1, 4), P), Node(YNWrBX8w6YwPcnuLYwteJA, (1, 5), P)), (Node(YNWrBX8w6YwPcnuLYwteJA, (1, 5), P), Node(ABGXREiYvdtqDDN3HFaWwB, (1, 6), P)), (Node(ABGXREiYvdtqDDN3HFaWwB, (1, 6), P), Node(XQ5uCrbivtNXbyJtwtJirs, (1, 7), P)), (Node(XQ5uCrbivtNXbyJtwtJirs, (1, 7), P), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P)), (Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P), Node(Msmr9nNPXxEGgYFRCwLQnh, (3, 1), P)), (Node(Msmr9nNPXxEGgYFRCwLQnh, (3, 1), P), Node(nj4T2NZSGmFnuGWqBgf6kn, (4, 1), P)), (Node(nj4T2NZSGmFnuGWqBgf6kn, (4, 1), P), Node(gRrLuGysZCKdBxPLKh9n5M, (5, 1), P)), (Node(gRrLuGysZCKdBxPLKh9n5M, (5, 1), P), Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P))])

In [18]:
buildings.nodes, pavements.nodes


(NodeView((Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G), Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S), Node(GYR9pCcwYY2XY9TxXdDCnM, (7, 1), H), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P), Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P), Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P), Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P))),
 NodeView((Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P), Node(PHNeqBRRyKVRqxzooC5AFa, (1, 3), P), Node(6yHugKLTsZJ3BKCMmz869Q, (1, 4), P), Node(YNWrBX8w6YwPcnuLYwteJA, (1, 5), P), Node(ABGXREiYvdtqDDN3HFaWwB, (1, 6), P), Node(XQ5uCrbivtNXbyJtwtJirs, (1, 7), P), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P), Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P), Node(Msmr9nNPXxEGgYFRCwLQnh, (3, 1), P), Node(nj4T2NZSGmFnuGWqBgf6kn, (4, 1), P), Node(gRrLuGysZCKdBxPLKh9n5M, (5, 1), P), Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P))))

In [21]:
buildings.nodes.items()

ItemsView(NodeView((Node(gAKyfiB2gvhk3M4zesJCHX, (1, 1), G), Node(SmNc8aqKQqR6HEu6U3mEuB, (1, 9), S), Node(GYR9pCcwYY2XY9TxXdDCnM, (7, 1), H), Node(8rKy9ztEH7z9AMpokagfkJ, (1, 8), P), Node(dNACFqpPdwNp8qoxYauP82, (1, 2), P), Node(23wHU3yz8zvwQ8hRzfpjpH, (2, 1), P), Node(9HqtLUwQb3PHsTgmGHaZA7, (6, 1), P))))

In [ ]:
nx.shortest_path_length(pavements, source=home, target=school)

NodeNotFound: Either source Node(FzSdFfPQywxz8c7uqCUMdX, (7, 1), H) or target Node(ciPiYGF5ktSxQFPqcm7WU5, (1, 9), S) is not in G